In [1]:
import mediapipe as mp
import os
import tensorflow as tf
import cv2
import numpy as np

def run_detection():
    # --- Load the saved model and class names ---
    model_path = 'asl_model.h5'
    class_names_path = 'class_names.txt'
    IMG_SIZE = (64, 64) # Ensure this matches the training configuration

    if not os.path.exists(model_path):
        print(f"Error: Model file not found at '{model_path}'")
        print("Please run the training cells in Part 1 to generate the model file.")
        return
    else:
        # Load the trained model
        model = tf.keras.models.load_model(model_path)
        print(f"Model '{model_path}' loaded successfully.")

        if not os.path.exists(class_names_path):
            print(f"Error: Class names file not found at '{class_names_path}'")
            return
        else:
            # Load the class names
            with open(class_names_path, 'r') as f:
                class_names = [line.strip() for line in f.readlines()]
            print(f"Class names loaded successfully: {class_names}")

    # --- Real-time Webcam Hand Sign Detection with Mediapipe Landmarks---
    
    # Initialize Mediapipe Hands
    mp_hands = mp.solutions.hands
    hands = mp_hands.Hands(static_image_mode=False, max_num_hands=1, min_detection_confidence=0.5)
    mp_drawing = mp.solutions.drawing_utils

    print("\n--- Starting Webcam for Real-time Detection ---")
    print("Place your hand inside the green box.")
    print("Press 'q' to quit the webcam window.")

    cap = cv2.VideoCapture(0)

    if not cap.isOpened():
        print("Error: Could not open webcam.")
        exit()

    # Define the Region of Interest (ROI) box
    box_size = 300
    ret, frame = cap.read()
    if not ret:
        print("Error: Can't read initial frame.")
        exit()
    height, width, _ = frame.shape

    x1 = int((width - box_size) / 2)
    y1 = int((height - box_size) / 2)
    x2 = x1 + box_size
    y2 = y1 + box_size

    while True:
        ret, frame = cap.read()
        if not ret:
            print("Error: Can't receive frame. Exiting ...")
            break

        frame = cv2.flip(frame, 1)

        # Extract the ROI for both prediction and landmark detection
        roi = frame[y1:y2, x1:x2]

        # --- Preprocess the ROI for the model ---
        rgb_roi_model = cv2.cvtColor(roi, cv2.COLOR_BGR2RGB)
        resized_roi = cv2.resize(rgb_roi_model, IMG_SIZE)
        img_array = tf.keras.utils.img_to_array(resized_roi)
        img_array = tf.expand_dims(img_array, 0)

        # --- Make a prediction on the ROI ---
        predictions = model.predict(img_array, verbose=0)
        predicted_index = np.argmax(predictions[0])
        predicted_class = class_names[predicted_index]
        confidence = 100 * np.max(predictions[0])

        # --- Process the ROI with Mediapipe to find hand landmarks ---
        rgb_roi_mp = cv2.cvtColor(roi, cv2.COLOR_BGR2RGB)
        results = hands.process(rgb_roi_mp)

        # --- NEW: Draw the hand landmarks on the ROI ---
        #if results.multi_hand_landmarks:
         #   for hand_landmarks in results.multi_hand_landmarks:
          #      mp_drawing.draw_landmarks(
          #          roi,  # Draw on the original ROI, not the RGB version
           #         hand_landmarks,
            #        mp_hands.HAND_CONNECTIONS,
             #       mp_drawing.DrawingSpec(color=(121, 22, 76), thickness=2, circle_radius=4), # Landmark style
              #      mp_drawing.DrawingSpec(color=(250, 44, 250), thickness=2, circle_radius=2)  # Connection style
               # )

        # --- Display the prediction and the ROI box on the main frame ---
        display_text = f"Prediction: {predicted_class} ({confidence:.1f}%)"
        cv2.putText(frame, display_text, (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2, cv2.LINE_AA)
        cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 255, 0), 2)

        # Display the resulting frame
        cv2.imshow('ASL Hand Sign Detection (Press Q to Quit)', frame)

        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

    cap.release()
    cv2.destroyAllWindows()
    hands.close()
    print("--- Webcam feed closed ---")

# To run the real-time detection, uncomment the line below and run this script
# Be sure you have trained and saved the model first by running train_model()
run_detection()

AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

Model 'asl_model.h5' loaded successfully.
Class names loaded successfully: ['A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z', 'del', 'nothing', 'space']

--- Starting Webcam for Real-time Detection ---
Place your hand inside the green box.
Press 'q' to quit the webcam window.
--- Webcam feed closed ---
